# Tool Search with Embeddings: Client-Side Implementation

This notebook demonstrates how to implement a client-side **Tool Search Tool** that enables Claude to work with hundreds or thousands of tools without context bloat. Instead of providing all tools upfront, we give Claude a single `tool_search` tool that returns relevant tool definitions on demand using semantic search.

## The Problem

When working with many tools, including all tool definitions in the initial request:
- Consumes excessive context window space
- Increases latency and costs
- Makes it harder for Claude to find the right tool
- Doesn't scale beyond ~100 tools

## The Solution

The Tool Search Tool pattern solves this by:
1. Providing Claude with only a `tool_search` tool initially
2. When Claude needs a tool, it searches using natural language
3. We return relevant tool definitions dynamically in the tool result
4. Claude can then use those discovered tools

## Implementation Approach

In this notebook, we will:
1. Create a library of diverse tools
2. Embed each tool using an open-source SentenceTransformer model (all-MiniLM-L6-v2)
3. Implement semantic search using cosine similarity
4. Handle tool search requests from Claude
5. Return tool definitions in tool results (using the shipped feature from PR #149961)

This follows the same pattern validated in production and demonstrated in the basic curl test.

## Setup

First, let's install the required packages and set up your API key. You'll need:
- [Claude API key](https://docs.anthropic.com/claude/reference/getting-started-with-the-api)

For embeddings, we'll use the open-source SentenceTransformer model `all-MiniLM-L6-v2`, which runs locally and doesn't require an API key.

In [ ]:
%pip install anthropic sentence-transformers numpy

In [ ]:
# Insert your API key here
ANTHROPIC_API_KEY = "<YOUR_ANTHROPIC_API_KEY>"

In [ ]:
import anthropic
from sentence_transformers import SentenceTransformer
import numpy as np
import json
from typing import List, Dict, Any, Tuple

# Initialize Claude client
claude_client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
    base_url="https://api-staging.anthropic.com",  # todo: remove this in case we ever publish this cookbook
)

# Load the SentenceTransformer model
# all-MiniLM-L6-v2 is a lightweight model with 384 dimensional embeddings
# It will be downloaded from HuggingFace on first use
print("Loading SentenceTransformer model...")
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

print("✓ Clients initialized successfully")

## Define Tool Library

Let's create a diverse library of 20 tools across different categories. In a real implementation, you might have hundreds or thousands of tools.

In [ ]:
# Define our tool library
TOOL_LIBRARY = [
    # Weather & Time
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The unit of temperature"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_forecast",
        "description": "Get the weather forecast for multiple days ahead",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "The city and state"},
                "days": {"type": "number", "description": "Number of days to forecast (1-10)"}
            },
            "required": ["location", "days"]
        }
    },
    {
        "name": "get_timezone",
        "description": "Get the current timezone and time for a location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "City name or timezone identifier"}
            },
            "required": ["location"]
        }
    },
    # Finance
    {
        "name": "get_stock_price",
        "description": "Get the current stock price and market data for a given ticker symbol",
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "Stock ticker symbol (e.g., AAPL, GOOGL)"},
                "include_history": {"type": "boolean", "description": "Include historical data"}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "convert_currency",
        "description": "Convert an amount from one currency to another using current exchange rates",
        "input_schema": {
            "type": "object",
            "properties": {
                "amount": {"type": "number", "description": "Amount to convert"},
                "from_currency": {"type": "string", "description": "Source currency code (e.g., USD)"},
                "to_currency": {"type": "string", "description": "Target currency code (e.g., EUR)"}
            },
            "required": ["amount", "from_currency", "to_currency"]
        }
    },
    {
        "name": "calculate_compound_interest",
        "description": "Calculate compound interest for investments over time",
        "input_schema": {
            "type": "object",
            "properties": {
                "principal": {"type": "number", "description": "Initial investment amount"},
                "rate": {"type": "number", "description": "Annual interest rate (as percentage)"},
                "years": {"type": "number", "description": "Number of years"},
                "frequency": {"type": "string", "enum": ["daily", "monthly", "quarterly", "annually"]}
            },
            "required": ["principal", "rate", "years"]
        }
    },
    # Communication
    {
        "name": "send_email",
        "description": "Send an email message to one or more recipients",
        "input_schema": {
            "type": "object",
            "properties": {
                "to": {"type": "array", "items": {"type": "string"}, "description": "Recipient email addresses"},
                "subject": {"type": "string", "description": "Email subject"},
                "body": {"type": "string", "description": "Email body content"},
                "cc": {"type": "array", "items": {"type": "string"}, "description": "CC recipients"}
            },
            "required": ["to", "subject", "body"]
        }
    },
    {
        "name": "send_sms",
        "description": "Send an SMS text message to a phone number",
        "input_schema": {
            "type": "object",
            "properties": {
                "phone_number": {"type": "string", "description": "Phone number with country code"},
                "message": {"type": "string", "description": "Text message content"}
            },
            "required": ["phone_number", "message"]
        }
    },
    {
        "name": "schedule_meeting",
        "description": "Schedule a calendar meeting with attendees",
        "input_schema": {
            "type": "object",
            "properties": {
                "title": {"type": "string", "description": "Meeting title"},
                "attendees": {"type": "array", "items": {"type": "string"}, "description": "Email addresses"},
                "start_time": {"type": "string", "description": "ISO 8601 datetime"},
                "duration_minutes": {"type": "number", "description": "Meeting duration"},
                "location": {"type": "string", "description": "Meeting location or video link"}
            },
            "required": ["title", "attendees", "start_time", "duration_minutes"]
        }
    },
    # Data & Files
    {
        "name": "read_file",
        "description": "Read the contents of a file from the filesystem",
        "input_schema": {
            "type": "object",
            "properties": {
                "file_path": {"type": "string", "description": "Path to the file"},
                "encoding": {"type": "string", "description": "File encoding (default: utf-8)"}
            },
            "required": ["file_path"]
        }
    },
    {
        "name": "search_files",
        "description": "Search for files in a directory matching a pattern",
        "input_schema": {
            "type": "object",
            "properties": {
                "directory": {"type": "string", "description": "Directory to search"},
                "pattern": {"type": "string", "description": "Search pattern or glob"},
                "recursive": {"type": "boolean", "description": "Search subdirectories"}
            },
            "required": ["directory", "pattern"]
        }
    },
    {
        "name": "create_spreadsheet",
        "description": "Create a new spreadsheet with data",
        "input_schema": {
            "type": "object",
            "properties": {
                "filename": {"type": "string", "description": "Name of the spreadsheet file"},
                "data": {"type": "array", "description": "Array of row data"},
                "headers": {"type": "array", "items": {"type": "string"}, "description": "Column headers"}
            },
            "required": ["filename", "data"]
        }
    },
    # Travel
    {
        "name": "search_flights",
        "description": "Search for available flights between two locations",
        "input_schema": {
            "type": "object",
            "properties": {
                "origin": {"type": "string", "description": "Origin airport code or city"},
                "destination": {"type": "string", "description": "Destination airport code or city"},
                "date": {"type": "string", "description": "Departure date (YYYY-MM-DD)"},
                "return_date": {"type": "string", "description": "Return date (optional)"},
                "passengers": {"type": "number", "description": "Number of passengers"}
            },
            "required": ["origin", "destination", "date"]
        }
    },
    {
        "name": "search_hotels",
        "description": "Search for hotels in a location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "City or area name"},
                "check_in": {"type": "string", "description": "Check-in date (YYYY-MM-DD)"},
                "check_out": {"type": "string", "description": "Check-out date (YYYY-MM-DD)"},
                "guests": {"type": "number", "description": "Number of guests"},
                "min_rating": {"type": "number", "description": "Minimum star rating"}
            },
            "required": ["location", "check_in", "check_out", "guests"]
        }
    },
    {
        "name": "calculate_distance",
        "description": "Calculate the distance between two locations",
        "input_schema": {
            "type": "object",
            "properties": {
                "origin": {"type": "string", "description": "Starting location"},
                "destination": {"type": "string", "description": "Ending location"},
                "unit": {"type": "string", "enum": ["km", "miles"], "description": "Distance unit"}
            },
            "required": ["origin", "destination"]
        }
    },
    # Shopping
    {
        "name": "search_products",
        "description": "Search for products across online retailers",
        "input_schema": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Product search query"},
                "category": {"type": "string", "description": "Product category filter"},
                "max_price": {"type": "number", "description": "Maximum price"},
                "min_rating": {"type": "number", "description": "Minimum customer rating"}
            },
            "required": ["query"]
        }
    },
    {
        "name": "track_order",
        "description": "Track the status of an order using tracking number",
        "input_schema": {
            "type": "object",
            "properties": {
                "tracking_number": {"type": "string", "description": "Order tracking number"},
                "carrier": {"type": "string", "description": "Shipping carrier (optional)"}
            },
            "required": ["tracking_number"]
        }
    },
    # Productivity
    {
        "name": "create_task",
        "description": "Create a new task in the task management system",
        "input_schema": {
            "type": "object",
            "properties": {
                "title": {"type": "string", "description": "Task title"},
                "description": {"type": "string", "description": "Task description"},
                "due_date": {"type": "string", "description": "Due date (YYYY-MM-DD)"},
                "priority": {"type": "string", "enum": ["low", "medium", "high"], "description": "Task priority"},
                "assignee": {"type": "string", "description": "Person assigned to task"}
            },
            "required": ["title"]
        }
    },
    {
        "name": "set_reminder",
        "description": "Set a reminder for a specific date and time",
        "input_schema": {
            "type": "object",
            "properties": {
                "message": {"type": "string", "description": "Reminder message"},
                "datetime": {"type": "string", "description": "When to remind (ISO 8601 datetime)"},
                "recurring": {"type": "boolean", "description": "Whether reminder repeats"}
            },
            "required": ["message", "datetime"]
        }
    }
]

print(f"✓ Defined {len(TOOL_LIBRARY)} tools in the library")

## Create Tool Embeddings

Now we'll convert each tool into an embedding vector. We create a text representation of each tool by combining its name, description, and parameter information, then embed it using the SentenceTransformer `all-MiniLM-L6-v2` model.

In [ ]:
def tool_to_text(tool: Dict[str, Any]) -> str:
    """
    Convert a tool definition into a text representation for embedding.
    Combines the tool name, description, and parameter information.
    """
    text_parts = [
        f"Tool: {tool['name']}",
        f"Description: {tool['description']}"
    ]
    
    # Add parameter information
    if 'input_schema' in tool and 'properties' in tool['input_schema']:
        params = tool['input_schema']['properties']
        param_descriptions = []
        for param_name, param_info in params.items():
            param_desc = param_info.get('description', '')
            param_type = param_info.get('type', '')
            param_descriptions.append(f"{param_name} ({param_type}): {param_desc}")
        
        if param_descriptions:
            text_parts.append("Parameters: " + ", ".join(param_descriptions))
    
    return "\n".join(text_parts)

# Test with one tool
sample_text = tool_to_text(TOOL_LIBRARY[0])
print("Sample tool text representation:")
print(sample_text)

In [ ]:
# Create embeddings for all tools
print("Creating embeddings for all tools...")

tool_texts = [tool_to_text(tool) for tool in TOOL_LIBRARY]

# Embed all tools at once using SentenceTransformer
# The model returns normalized embeddings by default
tool_embeddings = embedding_model.encode(tool_texts, convert_to_numpy=True)

print(f"✓ Created embeddings with shape: {tool_embeddings.shape}")
print(f"  - {tool_embeddings.shape[0]} tools")
print(f"  - {tool_embeddings.shape[1]} dimensions per embedding")

## Implement Tool Search

Now let's implement the core search functionality. When Claude asks for tools, we'll:
1. Embed the search query
2. Calculate cosine similarity with all tool embeddings
3. Return the top N most relevant tools

In [ ]:
def search_tools(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    """
    Search for tools using semantic similarity.
    
    Args:
        query: Natural language description of what tool is needed
        top_k: Number of top tools to return
    
    Returns:
        List of tool definitions most relevant to the query
    """
    # Embed the query using SentenceTransformer
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)
    
    # Calculate cosine similarity using dot product
    # SentenceTransformer returns normalized embeddings, so dot product = cosine similarity
    similarities = np.dot(tool_embeddings, query_embedding)
    
    # Get top k indices
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    # Return the corresponding tools with their scores
    results = []
    for idx in top_indices:
        results.append({
            'tool': TOOL_LIBRARY[idx],
            'similarity_score': float(similarities[idx])
        })
    
    return results

# Test the search function
test_query = "I need to check the weather"
test_results = search_tools(test_query, top_k=3)

print(f"Search query: '{test_query}'\n")
print("Top 3 matching tools:")
for i, result in enumerate(test_results, 1):
    tool_name = result['tool']['name']
    score = result['similarity_score']
    print(f"{i}. {tool_name} (similarity: {score:.3f})")

## Define the tool_search Tool

This is the meta-tool that Claude will use to discover other tools. It's the only tool we provide initially.

In [ ]:
# The tool_search tool definition
TOOL_SEARCH_DEFINITION = {
    "name": "tool_search",
    "description": "Search for available tools that can help with a task. Returns tool definitions for matching tools. Use this when you need a tool but don't have it available yet.",
    "input_schema": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Natural language description of what kind of tool you need (e.g., 'weather information', 'send email', 'search for flights')"
            },
            "top_k": {
                "type": "number",
                "description": "Number of tools to return (default: 5)"
            }
        },
        "required": ["query"]
    }
}

print("✓ Tool search definition created")

## Handle Tool Invocations

Now we'll implement the handler that processes tool calls from Claude. When Claude calls `tool_search`, we return the matching tool definitions in the tool result.

In [ ]:
def handle_tool_search(query: str, top_k: int = 5) -> Dict[str, Any]:
    """
    Handle a tool_search invocation and return tool definitions.
    
    Returns a tool result content block with discovered tools.
    """
    # Search for relevant tools
    results = search_tools(query, top_k=top_k)
    
    # Extract just the tool definitions (without scores)
    discovered_tools = [result['tool'] for result in results]
    
    print(f"\n🔍 Tool search: '{query}'")
    print(f"   Found {len(discovered_tools)} tools:")
    for i, result in enumerate(results, 1):
        print(f"   {i}. {result['tool']['name']} (similarity: {result['similarity_score']:.3f})")
    
    # Return in the format expected by Claude
    return {
        "type": "tools",
        "tools": discovered_tools
    }

# Test the handler
test_result = handle_tool_search("weather forecast", top_k=3)
print(f"\nReturned {len(test_result['tools'])} tool definitions")

## Implement Conversation Loop

Now let's put it all together! We'll create a conversation loop that:
1. Starts with only the `tool_search` tool
2. Handles tool invocations from Claude
3. Returns discovered tools in tool results
4. Allows Claude to use those tools

In [ ]:
import random
from datetime import datetime, timedelta

def mock_tool_execution(tool_name: str, tool_input: Dict[str, Any]) -> str:
    """
    Generate realistic mock responses for tool executions.
    
    Args:
        tool_name: Name of the tool being executed
        tool_input: Input parameters for the tool
    
    Returns:
        Mock response string appropriate for the tool
    """
    # Weather tools
    if tool_name == "get_weather":
        location = tool_input.get("location", "Unknown")
        unit = tool_input.get("unit", "fahrenheit")
        temp = random.randint(15, 30) if unit == "celsius" else random.randint(60, 85)
        conditions = random.choice(["sunny", "partly cloudy", "cloudy", "rainy"])
        return json.dumps({
            "location": location,
            "temperature": temp,
            "unit": unit,
            "conditions": conditions,
            "humidity": random.randint(40, 80),
            "wind_speed": random.randint(5, 20)
        })
    
    elif tool_name == "get_forecast":
        location = tool_input.get("location", "Unknown")
        days = int(tool_input.get("days", 5))
        forecast = []
        for i in range(days):
            date = (datetime.now() + timedelta(days=i)).strftime("%Y-%m-%d")
            forecast.append({
                "date": date,
                "high": random.randint(20, 30),
                "low": random.randint(10, 20),
                "conditions": random.choice(["sunny", "cloudy", "rainy", "partly cloudy"])
            })
        return json.dumps({"location": location, "forecast": forecast})
    
    elif tool_name == "get_timezone":
        location = tool_input.get("location", "Unknown")
        return json.dumps({
            "location": location,
            "timezone": "UTC+9",
            "current_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "utc_offset": "+09:00"
        })
    
    # Finance tools
    elif tool_name == "get_stock_price":
        ticker = tool_input.get("ticker", "UNKNOWN")
        return json.dumps({
            "ticker": ticker,
            "price": round(random.uniform(100, 500), 2),
            "change": round(random.uniform(-5, 5), 2),
            "change_percent": round(random.uniform(-2, 2), 2),
            "volume": random.randint(1000000, 10000000),
            "market_cap": f"${random.randint(100, 1000)}B"
        })
    
    elif tool_name == "convert_currency":
        amount = tool_input.get("amount", 0)
        from_currency = tool_input.get("from_currency", "USD")
        to_currency = tool_input.get("to_currency", "EUR")
        # Mock exchange rate
        rate = random.uniform(0.8, 1.2)
        converted = round(amount * rate, 2)
        return json.dumps({
            "original_amount": amount,
            "from_currency": from_currency,
            "to_currency": to_currency,
            "exchange_rate": round(rate, 4),
            "converted_amount": converted
        })
    
    elif tool_name == "calculate_compound_interest":
        principal = tool_input.get("principal", 0)
        rate = tool_input.get("rate", 0)
        years = tool_input.get("years", 0)
        frequency = tool_input.get("frequency", "monthly")
        
        # Calculate compound interest
        n_map = {"daily": 365, "monthly": 12, "quarterly": 4, "annually": 1}
        n = n_map.get(frequency, 12)
        final_amount = principal * (1 + rate/100/n) ** (n * years)
        interest_earned = final_amount - principal
        
        return json.dumps({
            "principal": principal,
            "rate": rate,
            "years": years,
            "compounding_frequency": frequency,
            "final_amount": round(final_amount, 2),
            "interest_earned": round(interest_earned, 2)
        })
    
    # Communication tools
    elif tool_name == "send_email":
        return json.dumps({
            "status": "sent",
            "message_id": f"msg_{random.randint(100000, 999999)}",
            "recipients": tool_input.get("to", []),
            "subject": tool_input.get("subject", ""),
            "timestamp": datetime.now().isoformat()
        })
    
    elif tool_name == "send_sms":
        return json.dumps({
            "status": "delivered",
            "message_id": f"sms_{random.randint(100000, 999999)}",
            "phone_number": tool_input.get("phone_number", ""),
            "timestamp": datetime.now().isoformat()
        })
    
    elif tool_name == "schedule_meeting":
        return json.dumps({
            "status": "scheduled",
            "meeting_id": f"meet_{random.randint(100000, 999999)}",
            "title": tool_input.get("title", ""),
            "attendees": tool_input.get("attendees", []),
            "start_time": tool_input.get("start_time", ""),
            "calendar_link": f"https://calendar.example.com/event/{random.randint(1000, 9999)}"
        })
    
    # File tools
    elif tool_name == "read_file":
        return json.dumps({
            "file_path": tool_input.get("file_path", ""),
            "content": "This is mock file content. In a real implementation, this would contain the actual file contents.",
            "size_bytes": random.randint(100, 10000),
            "encoding": tool_input.get("encoding", "utf-8")
        })
    
    elif tool_name == "search_files":
        num_results = random.randint(3, 10)
        files = [f"/path/to/file_{i}.txt" for i in range(num_results)]
        return json.dumps({
            "directory": tool_input.get("directory", ""),
            "pattern": tool_input.get("pattern", ""),
            "matches": files,
            "count": num_results
        })
    
    elif tool_name == "create_spreadsheet":
        return json.dumps({
            "status": "created",
            "filename": tool_input.get("filename", "spreadsheet.xlsx"),
            "rows": len(tool_input.get("data", [])),
            "file_path": f"/documents/{tool_input.get('filename', 'spreadsheet.xlsx')}"
        })
    
    # Travel tools
    elif tool_name == "search_flights":
        origin = tool_input.get("origin", "Unknown")
        destination = tool_input.get("destination", "Unknown")
        flights = []
        for i in range(3):
            flights.append({
                "flight_number": f"{random.choice(['AA', 'UA', 'DL'])}{random.randint(100, 999)}",
                "departure": f"{random.randint(6, 20):02d}:{random.choice(['00', '15', '30', '45'])}",
                "arrival": f"{random.randint(8, 22):02d}:{random.choice(['00', '15', '30', '45'])}",
                "price": random.randint(200, 800),
                "airline": random.choice(["American Airlines", "United", "Delta"])
            })
        return json.dumps({
            "origin": origin,
            "destination": destination,
            "date": tool_input.get("date", ""),
            "flights": flights
        })
    
    elif tool_name == "search_hotels":
        location = tool_input.get("location", "Unknown")
        hotels = []
        for i in range(3):
            hotels.append({
                "name": f"{random.choice(['Grand', 'Royal', 'Plaza', 'Comfort'])} {random.choice(['Hotel', 'Inn', 'Suites'])}",
                "rating": round(random.uniform(3.5, 5.0), 1),
                "price_per_night": random.randint(80, 300),
                "amenities": random.sample(["WiFi", "Pool", "Gym", "Restaurant", "Spa", "Parking"], 3)
            })
        return json.dumps({
            "location": location,
            "check_in": tool_input.get("check_in", ""),
            "check_out": tool_input.get("check_out", ""),
            "hotels": hotels
        })
    
    elif tool_name == "calculate_distance":
        origin = tool_input.get("origin", "Unknown")
        destination = tool_input.get("destination", "Unknown")
        unit = tool_input.get("unit", "miles")
        distance = random.randint(100, 5000)
        return json.dumps({
            "origin": origin,
            "destination": destination,
            "distance": distance,
            "unit": unit,
            "estimated_drive_time": f"{distance // 60} hours {distance % 60} minutes"
        })
    
    # Shopping tools
    elif tool_name == "search_products":
        query = tool_input.get("query", "")
        products = []
        for i in range(5):
            products.append({
                "name": f"{query} - Model {i+1}",
                "price": round(random.uniform(20, 200), 2),
                "rating": round(random.uniform(3.5, 5.0), 1),
                "reviews": random.randint(10, 1000),
                "retailer": random.choice(["Amazon", "Target", "Walmart", "Best Buy"])
            })
        return json.dumps({
            "query": query,
            "results": products,
            "count": len(products)
        })
    
    elif tool_name == "track_order":
        tracking_number = tool_input.get("tracking_number", "")
        statuses = [
            {"date": "2025-01-10", "status": "Order Placed", "location": "Online"},
            {"date": "2025-01-11", "status": "Shipped", "location": "Warehouse - Seattle, WA"},
            {"date": "2025-01-12", "status": "In Transit", "location": "Distribution Center - Denver, CO"},
            {"date": "2025-01-13", "status": "Out for Delivery", "location": "Local Facility"}
        ]
        return json.dumps({
            "tracking_number": tracking_number,
            "carrier": tool_input.get("carrier", "UPS"),
            "status": "Out for Delivery",
            "estimated_delivery": "2025-01-13",
            "history": statuses
        })
    
    # Productivity tools
    elif tool_name == "create_task":
        return json.dumps({
            "status": "created",
            "task_id": f"task_{random.randint(1000, 9999)}",
            "title": tool_input.get("title", ""),
            "priority": tool_input.get("priority", "medium"),
            "due_date": tool_input.get("due_date", ""),
            "url": f"https://tasks.example.com/task/{random.randint(1000, 9999)}"
        })
    
    elif tool_name == "set_reminder":
        return json.dumps({
            "status": "scheduled",
            "reminder_id": f"reminder_{random.randint(1000, 9999)}",
            "message": tool_input.get("message", ""),
            "datetime": tool_input.get("datetime", ""),
            "recurring": tool_input.get("recurring", False)
        })
    
    # Default fallback
    else:
        return json.dumps({
            "status": "executed",
            "tool": tool_name,
            "message": f"Tool {tool_name} executed successfully with input: {json.dumps(tool_input)}"
        })

print("✓ Mock tool execution function created")

In [ ]:
def run_tool_search_conversation(user_message: str, max_turns: int = 5) -> None:
    """
    Run a conversation with Claude using the tool search pattern.
    
    Args:
        user_message: The initial user message
        max_turns: Maximum number of conversation turns
    """
    print(f"\n{'='*80}")
    print(f"USER: {user_message}")
    print(f"{'='*80}\n")
    
    # Initialize conversation with only tool_search available
    messages = [{"role": "user", "content": user_message}]
    
    for turn in range(max_turns):
        print(f"\n--- Turn {turn + 1} ---")
        
        # Call Claude with current message history
        response = claude_client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1024,
            tools=[TOOL_SEARCH_DEFINITION],
            messages=messages,
            # IMPORTANT: This beta header enables tool definitions in tool results
            extra_headers={"anthropic-beta": "tool-result-tool-definitions-2025-09-16"}
        )
        
        # Add assistant's response to messages
        messages.append({"role": "assistant", "content": response.content})
        
        # Check if we're done
        if response.stop_reason == "end_turn":
            print("\n✓ Conversation complete\n")
            # Print final response
            for block in response.content:
                if block.type == "text":
                    print(f"ASSISTANT: {block.text}")
            break
        
        # Handle tool uses
        if response.stop_reason == "tool_use":
            tool_results = []
            
            for block in response.content:
                if block.type == "text":
                    print(f"\nASSISTANT: {block.text}")
                    
                elif block.type == "tool_use":
                    tool_name = block.name
                    tool_input = block.input
                    tool_use_id = block.id
                    
                    print(f"\n🔧 Tool invocation: {tool_name}")
                    print(f"   Input: {json.dumps(tool_input, indent=2)}")
                    
                    if tool_name == "tool_search":
                        # Handle tool search
                        query = tool_input["query"]
                        top_k = tool_input.get("top_k", 5)
                        
                        # Get discovered tools
                        tool_content = handle_tool_search(query, top_k)
                        
                        # Create tool result with tool definitions
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": [tool_content]  # This contains the "type": "tools" block
                        })
                    else:
                        # Execute the discovered tool with mock data
                        mock_result = mock_tool_execution(tool_name, tool_input)
                        
                        # Print a preview of the result
                        if len(mock_result) > 150:
                            print(f"   ✅ Mock result: {mock_result[:150]}...")
                        else:
                            print(f"   ✅ Mock result: {mock_result}")
                        
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": mock_result
                        })
            
            # Add tool results to messages
            if tool_results:
                messages.append({"role": "user", "content": tool_results})
        else:
            print(f"\nUnexpected stop reason: {response.stop_reason}")
            break
    
    print(f"\n{'='*80}\n")

print("✓ Conversation loop implemented")

## Example 1: Weather Query

Let's test with a simple weather question. Claude should:
1. Call `tool_search` to find weather tools
2. Receive weather tool definitions in the result
3. Use one of the discovered tools

In [ ]:
run_tool_search_conversation(
    "What's the weather like in Tokyo?"
)

## Example 2: Multi-Category Query

Let's try a more complex query that requires tools from different categories.

In [ ]:
run_tool_search_conversation(
    "I need to plan a trip to Paris next month. Can you help me find flights and hotels?"
)

## Example 3: Shopping Query

Test with a shopping-related query.

In [ ]:
run_tool_search_conversation(
    "I need to track my Amazon order. The tracking number is 1Z999AA10123456784"
)

## Example 4: Finance Query

Test with a financial calculation.

In [ ]:
run_tool_search_conversation(
    "If I invest $10,000 at 5% annual interest for 10 years with monthly compounding, how much will I have?"
)